In [1]:
import numpy as np
import tensorflow as tf
from glob import glob
from pinn import get_network
from pinn.utils import atomic_dress, connect_dist_grad
from ase.collections import g2
from pinn.io import sparse_batch, load_qm9
from docs.notebooks.network_fns import preprocess_traintest_sets, train_and_evaluate_network, _generator, predict_energy
from pinn.optimizers import get
import time

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
physical_devices = tf.config.list_physical_devices()
tf.config.set_visible_devices(physical_devices[0], 'CPU')
tf.config.set_visible_devices([], 'GPU')

In [2]:
filelist = glob('/Users/miguelnavaharris/Project/QM9/*.xyz')
dataset = load_qm9(filelist, splits={'train':8, 'test':2})

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-08-01 02:00:11.734205: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-01 02:00:11.734285: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
for i in dataset['train']:
    print(i)
    break

{'elems': <tf.Tensor: shape=(19,), dtype=int32, numpy=
array([6, 6, 6, 7, 7, 6, 6, 8, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int32)>, 'coord': <tf.Tensor: shape=(19, 3), dtype=float32, numpy=
array([[ 0.12453828,  1.3613629 , -0.4989974 ],
       [-0.12656163,  0.10827199,  0.35607484],
       [-1.4915951 , -0.47954565,  0.15802407],
       [-2.393762  , -0.42546415,  1.1326509 ],
       [-3.503685  , -1.0339637 ,  0.6254009 ],
       [-4.7165313 , -1.1230104 ,  1.4049323 ],
       [-3.3004317 , -1.4496067 , -0.6441173 ],
       [-4.3128505 , -2.070327  , -1.2940868 ],
       [-2.0087893 , -1.1181273 , -0.99810946],
       [ 1.1302432 ,  1.7573676 , -0.32552806],
       [ 0.03032281,  1.1396557 , -1.5670043 ],
       [-0.59819144,  2.146699  , -0.25753975],
       [ 0.63609445, -0.6458262 ,  0.12352705],
       [-0.01897491,  0.3539689 ,  1.4169384 ],
       [-5.2713413 , -0.17838433,  1.3889738 ],
       [-5.34652   , -1.9130049 ,  0.9938531 ],
       [-4.453515  , -1.3603593 ,

2022-08-01 02:00:12.170589: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-08-01 02:00:12.170750: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [4]:
def get_traintest_sets(batch_size):
    train_set = dataset['train'].shuffle(1000).apply(sparse_batch(batch_size))
    test_set = dataset['test'].apply(sparse_batch(batch_size))
    return (train_set, test_set)

In [5]:
params = {'optimizer': {'class_name': 'Adam', 'config': {'learning_rate': {'class_name': 'ExponentialDecay', 'config': {'initial_learning_rate': 0.0003, 'decay_steps': 10000, 'decay_rate': 0.994}}, 'clipnorm': 0.01}}, 'network': {'name': 'PiNet', 'params': {'depth': 4, 'rc': 4.0, 'atom_types': [1, 6, 7, 8, 9]}}, 'model': {'name': 'potential_model', 'params': {'learning_rate': 0.001, 'e_scale': 1, 'e_dress': {}}}}

In [6]:
network = get_network(params['network'])

In [7]:
def preprocess_traintest_sets(train_set, test_set):
    for batch in train_set:
        batch = network.preprocess(batch)
        connect_dist_grad(batch)
    for batch in test_set:
        batch = network.preprocess(batch)
        connect_dist_grad(batch)

In [8]:
batch_size = 100
train_set, test_set = get_traintest_sets(batch_size)

In [9]:
for i in train_set:
    batch = i
    break

In [10]:
preprocessed_batch = network.preprocess(batch)

In [11]:
batch

{'elems': <tf.Tensor: shape=(4669,), dtype=int32, numpy=array([6, 6, 6, ..., 1, 1, 1], dtype=int32)>,
 'coord': <tf.Tensor: shape=(4669, 3), dtype=float32, numpy=
 array([[-6.4693406e-02,  1.5870639e+00,  4.2438656e-02],
        [ 9.1630202e-03,  4.9195502e-02,  5.1467914e-02],
        [ 7.7468097e-01, -4.6331716e-01,  1.2851182e+00],
        ...,
        [ 6.6229266e-01, -1.4575788e+00, -2.1914005e+00],
        [ 1.9549299e+00, -2.2261343e+00, -1.2430873e+00],
        [-3.1206596e+00, -9.2455101e-01, -2.8964984e-03]], dtype=float32)>,
 'e_data': <tf.Tensor: shape=(256,), dtype=float32, numpy=
 array([-419.20978, -401.8588 , -439.13382, -403.17117, -345.67416,
        -458.97852, -401.94016, -400.66592, -423.02658, -457.71317,
        -324.28998, -421.79355, -385.84146, -416.85675, -419.24103,
        -458.96118, -423.0138 , -363.84375, -416.84885, -347.76978,
        -303.46808, -440.24445, -440.23798, -401.9327 , -389.47876,
        -401.82077, -348.96768, -475.05264, -402.02304, -34

In [12]:
preprocessed_batch

{'elems': <tf.Tensor: shape=(4669,), dtype=int32, numpy=array([6, 6, 6, ..., 1, 1, 1], dtype=int32)>,
 'coord': <tf.Tensor: shape=(4669, 3), dtype=float32, numpy=
 array([[-6.4693406e-02,  1.5870639e+00,  4.2438656e-02],
        [ 9.1630202e-03,  4.9195502e-02,  5.1467914e-02],
        [ 7.7468097e-01, -4.6331716e-01,  1.2851182e+00],
        ...,
        [ 6.6229266e-01, -1.4575788e+00, -2.1914005e+00],
        [ 1.9549299e+00, -2.2261343e+00, -1.2430873e+00],
        [-3.1206596e+00, -9.2455101e-01, -2.8964984e-03]], dtype=float32)>,
 'e_data': <tf.Tensor: shape=(256,), dtype=float32, numpy=
 array([-419.20978, -401.8588 , -439.13382, -403.17117, -345.67416,
        -458.97852, -401.94016, -400.66592, -423.02658, -457.71317,
        -324.28998, -421.79355, -385.84146, -416.85675, -419.24103,
        -458.96118, -423.0138 , -363.84375, -416.84885, -347.76978,
        -303.46808, -440.24445, -440.23798, -401.9327 , -389.47876,
        -401.82077, -348.96768, -475.05264, -402.02304, -34

In [13]:
connect_dist_grad(preprocessed_batch)

In [14]:
preprocessed_batch

{'elems': <tf.Tensor: shape=(4669,), dtype=int32, numpy=array([6, 6, 6, ..., 1, 1, 1], dtype=int32)>,
 'coord': <tf.Tensor: shape=(4669, 3), dtype=float32, numpy=
 array([[-6.4693406e-02,  1.5870639e+00,  4.2438656e-02],
        [ 9.1630202e-03,  4.9195502e-02,  5.1467914e-02],
        [ 7.7468097e-01, -4.6331716e-01,  1.2851182e+00],
        ...,
        [ 6.6229266e-01, -1.4575788e+00, -2.1914005e+00],
        [ 1.9549299e+00, -2.2261343e+00, -1.2430873e+00],
        [-3.1206596e+00, -9.2455101e-01, -2.8964984e-03]], dtype=float32)>,
 'e_data': <tf.Tensor: shape=(256,), dtype=float32, numpy=
 array([-419.20978, -401.8588 , -439.13382, -403.17117, -345.67416,
        -458.97852, -401.94016, -400.66592, -423.02658, -457.71317,
        -324.28998, -421.79355, -385.84146, -416.85675, -419.24103,
        -458.96118, -423.0138 , -363.84375, -416.84885, -347.76978,
        -303.46808, -440.24445, -440.23798, -401.9327 , -389.47876,
        -401.82077, -348.96768, -475.05264, -402.02304, -34

In [15]:
preprocess_traintest_sets(train_set, test_set)

In [16]:
def train_and_evaluate_network(network=network, params=params, train_set=train_set, test_set=test_set, batch_size=256, epochs=1):


    # Instantiate an optimizer
    optimizer = get(params['optimizer'])
    # Define a loss function
    loss_fn = tf.keras.losses.mse
    # Define metrics
    train_loss_metric = tf.keras.metrics.MeanSquaredError()
    val_loss_metric = tf.keras.metrics.MeanSquaredError()
    train_err_metric = tf.keras.metrics.RootMeanSquaredError()
    val_err_metric = tf.keras.metrics.RootMeanSquaredError()
    

    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))
        start_time_epoch = time.time()
        hund_step_times = []
        val_errors = []


        # Iterate over the batches of the dataset.
        for step, batch in enumerate(train_set):

            # Open a GradientTape to record the operations run
            # during the forward pass, which enables auto-differentiation.
            with tf.GradientTape() as tape:

                # Run the forward pass of the layer.
                # The operations that the layer applies
                # to its inputs are going to be recorded
                # on the GradientTape.
                # start_time = time.time()

                pred = network(batch, training=True)  # Logits for this minibatch

                ind = batch['ind_1']
                nbatch = tf.reduce_max(ind)+1
                pred = tf.math.unsorted_segment_sum(pred, ind[:, 0], nbatch)
                e_data = batch['e_data']


                # Compute the loss value for this minibatch.
                loss_value = loss_fn(e_data, pred)

            # Use the gradient tape to automatically retrieve
            # the gradients of the trainable variables with respect to the loss.
            grads = tape.gradient(loss_value, network.trainable_weights)

            # Run one step of gradient descent by updating
            # the value of the variables to minimize the loss.
            optimizer.apply_gradients(zip(grads, network.trainable_weights))

            # Update the loss and error metrics
            train_loss_metric.update_state(e_data, pred)
            train_err_metric.update_state(e_data, pred)

            # end_time = time.time() - start_time
            # print('End time:', end_time)

            # Log every 100 batches.
            if step == 0:
                print(f"Initial loss (for one batch): {float(loss_value)}")
                print(f"Seen so far: {((step + 1) * batch_size)} molecules")



            elif step % 100 == 0:
                print(f"Training loss (for one batch) at step {step}: {float(loss_value)}")
                print(f"Seen so far: {((step + 1) * batch_size)} molecules")
                hund_step_times += [(time.time() - start_time_epoch)]
                print(f'Training time for 100 batches: {((hund_step_times[-1] - hund_step_times[-2]) if len(hund_step_times) > 1 else hund_step_times[-1])} s')



        print(f'Training time for epoch {epoch + 1}: {(time.time() - start_time_epoch)} s')
        

        # Run a validation loop at the end of each epoch
        print(f'Starting validation for epoch {(epoch + 1)}')

        for step, batch in enumerate(test_set):
            
            val_pred = network(batch, training=False)
            ind = batch['ind_1']
            nbatch = tf.reduce_max(ind)+1
            val_pred = tf.math.unsorted_segment_sum(val_pred, ind[:, 0], nbatch)
            e_data = batch['e_data']


            # Update val metrics
            val_loss_metric.update_state(e_data, val_pred)
            val_err_metric.update_state(e_data, val_pred)



        print(f"Time taken for epoch {epoch + 1}: {(time.time() - start_time_epoch)} s")

        # Display metrics at the end of each epoch
        train_err = train_err_metric.result()
        print(f"Training err over epoch {(epoch + 1)}: {float(train_err)}")
        val_err = val_err_metric.result()
        val_errors.append(float(val_err))
        print(f"Validation err for epoch {(epoch + 1)}: {float(val_err)}")
        
        # Reset training metrics at the end of each epoch 
        train_loss_metric.reset_states()
        val_loss_metric.reset_states()       
        train_err_metric.reset_states()
        val_err_metric.reset_states()

        print(val_errors)
        if epoch > 3 and val_errors[-4] <= min(val_errors[-3:]):
                    return 'Stopped.'

        

In [17]:
train_and_evaluate_network(batch_size=batch_size, epochs=1)


Start of epoch 0
Initial loss (for one batch): 144213.8125
Seen so far: 256 molecules
Training loss (for one batch) at step 100: 1389.159423828125
Seen so far: 25856 molecules
Training time for 100 batches: 60.52131676673889 s
Training loss (for one batch) at step 200: 882.26611328125
Seen so far: 51456 molecules
Training time for 100 batches: 71.90927934646606 s
Training loss (for one batch) at step 300: 568.4716186523438
Seen so far: 77056 molecules
Training time for 100 batches: 80.45273280143738 s


KeyboardInterrupt: 

In [11]:
def _generator(molecule):
        data = {'coord': molecule.positions,
                'ind_1': np.zeros([len(molecule), 1]),
                'elems': molecule.numbers}
        yield data

def predict_energy(molecule):
        '''Takes an ASE Atoms object and outputs PiNet's energy prediction'''
        dtype=tf.float32
        dtypes = {'coord': dtype, 'elems': tf.int32, 'ind_1': tf.int32}
        shapes = {'coord': [None, 3], 'elems': [None], 'ind_1': [None, 1]}

        pred_dataset = tf.data.Dataset.from_generator(lambda:_generator(molecule), dtypes, shapes)

        for molecule in pred_dataset:
                molecule = network.preprocess(molecule)
                pred = network(molecule, training=False)
                ind = molecule['ind_1']
                nbatch = tf.reduce_max(ind)+1
                energy_prediction = tf.math.unsorted_segment_sum(pred, ind[:, 0], nbatch)
                energy_prediction_numpy = energy_prediction.numpy()[0]
        return energy_prediction_numpy

In [12]:
next(_generator(g2['CH4']))

{'coord': array([[ 0.      ,  0.      ,  0.      ],
        [ 0.629118,  0.629118,  0.629118],
        [-0.629118, -0.629118,  0.629118],
        [ 0.629118, -0.629118, -0.629118],
        [-0.629118,  0.629118, -0.629118]]),
 'ind_1': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 'elems': array([6, 1, 1, 1, 1])}

In [19]:
predict_energy(g2['CH4'])

-18.794298

In [14]:
for i in train_set:
    print(i)
    a = i
    break

{'elems': <tf.Tensor: shape=(2333,), dtype=int32, numpy=array([7, 6, 6, ..., 1, 1, 1], dtype=int32)>, 'coord': <tf.Tensor: shape=(2333, 3), dtype=float32, numpy=
array([[ 0.06930045,  1.4449356 ,  0.17122535],
       [ 0.02267061,  0.07571542,  0.04459907],
       [-1.066454  , -0.76106334,  0.00750551],
       ...,
       [ 2.7930737 ,  0.19725555,  0.43674996],
       [ 2.0914726 ,  1.287523  ,  1.649632  ],
       [ 2.8104842 , -0.24979167,  2.157346  ]], dtype=float32)>, 'e_data': <tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-393.57846, -398.14343, -401.90503, -404.29816, -379.91382,
       -385.89896, -231.10631, -419.26816, -383.172  , -352.3668 ,
       -421.76602, -398.27084, -403.1459 , -437.80237, -420.39893,
       -388.2607 , -403.12582, -383.71817, -400.6426 , -492.00888,
       -423.04446, -455.13193, -398.1387 , -340.67972, -455.17688,
       -423.01047, -423.003  , -422.96106, -389.48065, -422.06546,
       -418.0412 , -388.30313, -475.03003, -421.8071 , -437.

In [15]:
b = network.preprocess(a)
connect_dist_grad(b)

In [16]:
b_pred = network(b)
ind = b['ind_1']
nbatch = tf.reduce_max(ind)+1
b_pred = tf.math.unsorted_segment_sum(b_pred, ind[:, 0], nbatch)
e_data = b['e_data']
e_data -= atomic_dress(b, params['model']['params']['e_dress'], dtype=b_pred.dtype)
e_data *= params['model']['params']['e_scale']

In [17]:
b_pred

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-381.86557, -409.28497, -398.29123, -421.11533, -394.76492,
       -395.13214, -235.63544, -416.60037, -391.5477 , -374.113  ,
       -450.78806, -396.96484, -419.63223, -430.58038, -424.91898,
       -398.59235, -410.30063, -374.46503, -414.35007, -441.37195,
       -426.23978, -459.04752, -412.0945 , -351.79498, -468.33627,
       -423.05798, -428.31393, -433.25558, -405.61154, -431.9105 ,
       -433.24655, -399.356  , -477.7931 , -416.9553 , -417.32074,
       -416.4994 , -433.56696, -414.82922, -427.69278, -422.05014,
       -372.68732, -388.9638 , -455.03983, -396.8519 , -435.05756,
       -377.2176 , -434.9269 , -369.26965, -301.68942, -330.91498,
       -468.44366, -436.28384, -369.51193, -413.43808, -422.75687,
       -482.6218 , -401.80673, -364.92288, -292.25894, -453.9418 ,
       -376.80652, -377.3983 , -412.63156, -368.18036, -503.91867,
       -358.40286, -316.86816, -470.128  , -420.11426, -414.24634,
       -341.88

In [18]:
e_data.numpy()

array([-393.57846, -398.14343, -401.90503, -404.29816, -379.91382,
       -385.89896, -231.10631, -419.26816, -383.172  , -352.3668 ,
       -421.76602, -398.27084, -403.1459 , -437.80237, -420.39893,
       -388.2607 , -403.12582, -383.71817, -400.6426 , -492.00888,
       -423.04446, -455.13193, -398.1387 , -340.67972, -455.17688,
       -423.01047, -423.003  , -422.96106, -389.48065, -422.06546,
       -418.0412 , -388.30313, -475.03003, -421.8071 , -437.86945,
       -416.7347 , -422.97955, -404.30325, -419.1927 , -417.99173,
       -352.31476, -387.09543, -439.0764 , -433.3102 , -422.93597,
       -378.74695, -440.34058, -365.07892, -309.72717, -314.273  ,
       -460.13208, -439.0638 , -394.59985, -403.06406, -417.95825,
       -456.34143, -389.48892, -351.13153, -319.56082, -439.0047 ,
       -365.95026, -353.56186, -387.12848, -351.16617, -494.92816,
       -343.29654, -309.71402, -461.3653 , -423.04935, -434.10315,
       -342.74634, -388.26227, -457.71317, -388.30676, -363.84

In [42]:
import sklearn as sk
from sklearn.metrics import mean_squared_error
mean_squared_error(e_data, b_pred, squared=False)

0.066774674